In [1]:
from netCDF4 import Dataset
import netCDF4
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from datetime import datetime, timezone
import nc_time_axis

atmos_epoch = datetime(1900, 1, 1, 0, 0, tzinfo=timezone.utc)

In [2]:
observational_geopotential = Dataset('../data/geopotential_height_500hPa_era5_6hourly_z0001_daymean_2019_beginAdjust.nc', mode='r')

observational_labels = Dataset('../data/labels/GTD_1979-2019_JJAextd_8.nc', mode='r')

In [3]:
long = observational_geopotential.variables['longitude']
lat = observational_geopotential.variables['latitude']
zg = observational_geopotential.variables['z_0001']
time_raw = observational_geopotential.variables["time"]
time = netCDF4.num2date(time_raw[:], time_raw.units, time_raw.calendar)

In [4]:
mean_mask = np.array(np.zeros((98, len(lat), len(long))))
zg_mean = np.array(np.zeros((len(time), len(lat), len(long))))

years = int(len(time) / 98) # number of days per year

for j in range(2):
    offset = 98 * j
    for i in range(98):
        mean_mask[i] = np.mean([zg[offset + i], zg[offset + 98 + i], zg[offset + 98 * 2 + i], zg[offset + 98 * 3 + i], zg[offset + 98 * 4 + i]])
        zg_mean[offset + i] = np.subtract(zg[offset + i], mean_mask[i])

for j in range(2, years-2):
    offset = 98 * j
    for i in range(98):
        mean_mask[i] = np.mean([zg[offset + i], zg[offset - 98 + i], zg[offset - 98 * 2 + i], zg[offset + 98 + i], zg[offset + 98 * 2 + i]])
        zg_mean[offset + i] = np.subtract(zg[offset + i], mean_mask[i])

for j in range(years-2, years):
    offset = 98 * j
    for i in range(98):
        if (offset + i >= len(time)): break
        mean_mask[i] = np.mean([zg[offset + i], zg[offset - 98 + i], zg[offset - 98 * 2 + i], zg[offset - 98 * 3 + i], zg[offset - 98 * 4 + i]])
        zg_mean[offset + i] = np.subtract(zg[offset + i], mean_mask[i])

print(zg_mean.shape)

(4018, 281, 1440)


In [6]:
import os

try:
    os.remove('../data/geopotential_height_500hPa_era5_6hourly_z0001_daymean_final.nc')
except:
    pass

ncfile = Dataset('../data/geopotential_height_500hPa_era5_6hourly_z0001_daymean_final.nc', mode='w', format='NETCDF4_CLASSIC')

ncfile.createDimension("time", None)
ncfile.createDimension("longitude", 1440)
ncfile.createDimension("latitude", 281)
ncfile.createDimension("day_range", 5)

for v_name, varin in observational_geopotential.variables.items():
    if v_name in ["time_bnds"]: continue
    fill_value = None
    
    if hasattr(varin, "_FillValue"):
        fill_value = varin._FillValue
    if v_name in ["z_0001"]:
        outVar = ncfile.createVariable(v_name, varin.datatype, ("time", "day_range", "longitude", "latitude"), fill_value=fill_value)
    else:
        outVar = ncfile.createVariable(v_name, varin.datatype, varin.dimensions, fill_value=fill_value)
    
    # Copy variable attributes
    outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs() if k not in ["_FillValue"]})

ncfile.variables["time"] = time_raw[:]
ncfile.variables["longitude"] = long[:]
ncfile.variables["latitude"] = lat[:]
ncfile.variables["day_range"] = [0, 1, 2, 3, 4]

result = np.zeros((len(time), 5, len(lat), len(long)))

for i in range(len(time)):
    print("working on " + str(i) + " of " + str(len(time)))
    if len(zg_mean) < 5:
        ncfile.variables["z_0001"][i] = np.array([zg_mean[0], zg_mean[0], zg_mean[0], zg_mean[0], zg_mean[0]])
    else:
        ncfile.variables["z_0001"][i] = np.array([zg_mean[0], zg_mean[0 + 1], zg_mean[0 + 2], zg_mean[0 + 3], zg_mean[0 + 4]])
    zg_mean = zg_mean[1:]
print(result.shape)
ncfile.close()

working on 0 of 4018
working on 1 of 4018
working on 2 of 4018
working on 3 of 4018
working on 4 of 4018
working on 5 of 4018
working on 6 of 4018
working on 7 of 4018
working on 8 of 4018
working on 9 of 4018
working on 10 of 4018
working on 11 of 4018
working on 12 of 4018
working on 13 of 4018
working on 14 of 4018
working on 15 of 4018
working on 16 of 4018
working on 17 of 4018
working on 18 of 4018
working on 19 of 4018
working on 20 of 4018
working on 21 of 4018
working on 22 of 4018
working on 23 of 4018
working on 24 of 4018
working on 25 of 4018
working on 26 of 4018
working on 27 of 4018
working on 28 of 4018
working on 29 of 4018
working on 30 of 4018
working on 31 of 4018
working on 32 of 4018
working on 33 of 4018
working on 34 of 4018
working on 35 of 4018
working on 36 of 4018
working on 37 of 4018
working on 38 of 4018
working on 39 of 4018
working on 40 of 4018
working on 41 of 4018
working on 42 of 4018
working on 43 of 4018
working on 44 of 4018
working on 45 of 401